In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 45)
pd.set_option('display.max_rows', 50)

In [3]:
item_master = pd.read_excel(r"data/item_master.xlsx", index_col=False, header=1)
material_history = pd.read_excel('data/Material Movement History.xlsx', index_col=False, header=1)

In [4]:
item_master

,Unnamed: 0,Plant,Material,Not Used,Description,Specipication,Material Type,Material Group,Unit,Unit Of Order,Unit Of Issue,Purchasing Group,Inventory Manager,Rounding Value,INT. Lead Time,EXT. Lead Time,Procurement,Special Procurement,Grade,MRP Type,Minimum LotSize,Maximum LotSize,Tolerance Usage,Backflash,QM Usage,Auto GR By CC,Automatic PO,GR Storage Location,GI Storage Location,Safety Stock Ind,Safety Stock Day,Safety Stock Qty,Basic Material,Tool Code,Packing Size,Lot Size,Volume,Assembly Scrap,QM Group,Status,Unnamed: 40,Create Date,Unnamed: 42,Change Date,Unnamed: 44
0,1,DYKV,BN61-08214A,0,40 P Bracket STAND (FERT),40/46E5000,FERT,BRACKET,PC,PC,PC,PC1,FPR,1,0,3,E,,A,ND,100,0,1,1,0,0,0,FPRE,FPRE,None,0,0,40/46 E5000,BN61-08214A,1,EX,0,0,OQC-PRE,Live,,08/01/2016 18:30:50,JUNG HYO IN,07/01/2021 10:32:47,IT_Kieu
1,2,DYKV,BN61-11437B,0,50 I FRAME-CHASSIS REAR TOP (FERT),50KU6000PC +GF10%,FERT,FRAME,PC,PC,PC,,FIN,1,0,0,E,,A,PD,2,0,1,1,0,0,0,FINJ,FINJ,None,0,0,,,1,EX,0,0,OQC-INJ,Live,06/14/2017,06/14/2017 15:32:00,thúy nga,01/24/2019 13:35:04,IT_Kieu
2,3,DYKV,BN61-11473B,0,50 I FRAME-CHASSIS REAR BOTTOM,50KU6000PC +GF,FERT,FRAME,PC,PC,PC,,FIN,1,0,0,E,,A,PD,2,0,1,1,0,0,0,FINJ,FINJ,None,0,0,,,1,EX,0,0,OQC-INJ,Live,06/15/2017,06/15/2017 16:18:20,thúy nga,01/24/2019 13:35:14,IT_Kieu
3,4,DYKV,BN61-12293B,0,48 P DP-BRACKET-STAND BODY (FERT),48Y15 F-LED J5100/5500,FERT,BRACKET,PC,PC,PC,,FPR,1,0,0,E,,A,PD,1,0,1,1,0,0,0,FPRE,FPRE,None,0,0,,,1,EX,0,0,OQC-PRE,Live,12/05/2017,12/05/2017 15:12:56,SALE_ DUY,08/24/2018 15:42:19,thúy nga
4,5,DYKV,BN61-13015B,0,43 I FRAME-CHASSIS REAR LEFT RIGHT(FERT),"43KU6000,P",FERT,DECO,PC,PC,PC,,WIL,500,0,0,E,,A,PD,500,0,1,1,0,0,0,WIAS,WIAS,None,0,0,43KU6000,,1,EX,0,0,OQC-INJ,Live,,08/01/2016 18:30:35,JUNG HYO IN,01/24/2019 13:35:23,IT_Kieu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8163,8164,DYKV,PB01-001600,0,Antistatic Bag (LDPE),"0.05*150*1600 (FM/BOT 65"", 55"")",ROH,SUBPART,EA,EA,EA,PC2,RSU,1,0,7,F,,A,ND,1,0,1,1,1,0,0,PACK,PACK,NONE,0,0,,,1,W1,0,0,IQC-SUB,Live,03/09/2017,03/09/2017 17:29:47,투이_THỦY,07/01/2017 13:17:10,KIM JIN SOO
8164,8165,DYKV,PE01-150155,0,EPE SHEET,1T*50cm*155cm,ROH,SUBPART,PC,PC,PC,PC1,RSU,1,0,7,F,,A,ND,1,0,1,1,0,0,0,PACK,PACK,NONE,0,0,,,1,W1,0,0,IQC-SUB,Live,07/24/2017,07/24/2017 16:18:34,thúy nga,,
8165,8166,DYKV,SH0140U,0,DW7010F-100C5,100*7*1T,ROH,SUBPART,PC,PC,PC,PC1,RSU,1,0,3,F,,A,ND,1,0,1,1,0,1,0,RSUB,RSUB,NONE,0,0,,,1,W1,0,0,,Live,10/17/2016,10/17/2016 08:52:46,IT LÝ,02/15/2017 15:12:54,KIM JIN SOO
8166,8167,DYKV,SMPL-00001,0,STEEL ROLL-EG SKELP(BLACK),JS-SECC 1600*230.00*C,ROH,STEEL,KG,KG,KG,PC2,RST,1,0,7,F,,A,ND,1,0,1,1,0,0,0,RSTL,RSTL,None,0,0,,,1,W1,0,0,,Live,09/20/2016,09/20/2016 08:51:03,투이_THỦY,02/20/2017 11:02:52,KIM JIN SOO


In [5]:
material_history

,Unnamed: 0,Plant,Entry Date,Material,Location,Stock Type,Movement Type,Posting Date,Unit,Quantity,Material Document No,Provide,Reference,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Previous Stock,Entry User
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Order Category,Order Data,Master Category,Master Data,Remark,NaN,NaN
1,1.0,DYKV,07/01/2021 23:59:24,BN60-00162C,WPAS,Unrestricted,261 (GI for production order),07/01/2021,M,-39.0,1003609213-16,,Production Order,6500297936,,,,783.06,MES_USER
2,2.0,DYKV,07/01/2021 23:59:24,BN61-13557P,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-60.0,1003609213-15,,Production Order,6500297936,,,,78043.00,MES_USER
3,3.0,DYKV,07/01/2021 23:59:24,BN61-16157A,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-30.0,1003609213-14,,Production Order,6500297936,,,,595.00,MES_USER
4,4.0,DYKV,07/01/2021 23:59:24,BN96-46032A,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-60.0,1003609213-13,,Production Order,6500297936,,,,1068.00,MES_USER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21668,4051.0,DYKV,07/01/2021 00:20:59,PZ64-04335A,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,50.0,1003607406-1,,Production Order,6500297552,,,,119.00,MES_USER
21669,4052.0,DYKV,07/01/2021 00:20:27,IZ64-04516B,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,30.0,1003607405-1,,Production Order,6500297647,,,,488.00,MES_USER
21670,4053.0,DYKV,07/01/2021 00:13:27,IZ64-04514A,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,80.0,1003607404-1,,Production Order,6500297548,,,,7050.00,MES_USER
21671,4054.0,DYKV,07/01/2021 00:12:41,BN96-50471A,FPRE,Unrestricted,101 (GR for Order),07/01/2021,PC,20.0,1003607403-1,,Production Order,6500297708,,,,465.00,MES_USER


In [6]:
print('Material Shape: ', material_history.shape)
print('Item master: ', item_master.shape)


Material Shape:  (21673, 19)
Item master:  (8168, 45)


In [7]:
material_history['Material'].value_counts()

BN02-00102B    905
BN61-13557P    839
BN61-17482A    413
0203-007086    406
BN61-13557U    386
              ... 
0203-007158      1
BN65-00010A      1
BN61-17611A      1
BN60-01155B      1
BN96-45678R      1
Name: Material, Length: 906, dtype: int64

In [8]:
temp_item_master = item_master[['Material', 'Material Type', 'Procurement']]

In [9]:
temp_item_master.loc[temp_item_master['Material'] == 'BN61-08214A']

,Material,Material Type,Procurement
0,BN61-08214A,FERT,E


In [10]:
step_one = pd.merge(material_history, temp_item_master, on="Material", how="left")

In [11]:
step_one

,Unnamed: 0,Plant,Entry Date,Material,Location,Stock Type,Movement Type,Posting Date,Unit,Quantity,Material Document No,Provide,Reference,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Previous Stock,Entry User,Material Type,Procurement
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Order Category,Order Data,Master Category,Master Data,Remark,NaN,NaN,NaN,NaN
1,1.0,DYKV,07/01/2021 23:59:24,BN60-00162C,WPAS,Unrestricted,261 (GI for production order),07/01/2021,M,-39.0,1003609213-16,,Production Order,6500297936,,,,783.06,MES_USER,ROH,F
2,2.0,DYKV,07/01/2021 23:59:24,BN61-13557P,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-60.0,1003609213-15,,Production Order,6500297936,,,,78043.00,MES_USER,ROH,F
3,3.0,DYKV,07/01/2021 23:59:24,BN61-16157A,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-30.0,1003609213-14,,Production Order,6500297936,,,,595.00,MES_USER,ROH,F
4,4.0,DYKV,07/01/2021 23:59:24,BN96-46032A,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-60.0,1003609213-13,,Production Order,6500297936,,,,1068.00,MES_USER,ROH,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21668,4051.0,DYKV,07/01/2021 00:20:59,PZ64-04335A,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,50.0,1003607406-1,,Production Order,6500297552,,,,119.00,MES_USER,HALB,E
21669,4052.0,DYKV,07/01/2021 00:20:27,IZ64-04516B,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,30.0,1003607405-1,,Production Order,6500297647,,,,488.00,MES_USER,HALB,E
21670,4053.0,DYKV,07/01/2021 00:13:27,IZ64-04514A,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,80.0,1003607404-1,,Production Order,6500297548,,,,7050.00,MES_USER,HALB,E
21671,4054.0,DYKV,07/01/2021 00:12:41,BN96-50471A,FPRE,Unrestricted,101 (GR for Order),07/01/2021,PC,20.0,1003607403-1,,Production Order,6500297708,,,,465.00,MES_USER,FERT,E


In [12]:
# step_one['Input'] = step_one['Quantity'] if step_one['Quantity'] > 0 else None
# step_one['Output'] = -step_one['Quantity'] if step_one['Quantity'] < 0 else None


In [13]:
step_one['Quantity']

0         NaN
1       -39.0
2       -60.0
3       -30.0
4       -60.0
         ... 
21668    50.0
21669    30.0
21670    80.0
21671    20.0
21672    90.0
Name: Quantity, Length: 21673, dtype: float64

In [14]:
def create_quantity(columns):
    if columns > 0:
        return [columns, 0]
    else:
        return [0, -columns]

In [15]:
step_one['Input'] = step_one['Quantity']
step_one['Output'] = step_one['Quantity']

In [16]:
# step_one['Input'] = [value for value in step_one['Input'] if step_one['Input'].all() > 0]

step_one.loc[0]

Unnamed: 0                          NaN
Plant                               NaN
Entry Date                          NaN
Material                            NaN
Location                            NaN
Stock Type                          NaN
Movement Type                       NaN
Posting Date                        NaN
Unit                                NaN
Quantity                            NaN
Material Document No                NaN
Provide                             NaN
Reference                Order Category
Unnamed: 13                  Order Data
Unnamed: 14             Master Category
Unnamed: 15                 Master Data
Unnamed: 16                      Remark
Previous Stock                      NaN
Entry User                          NaN
Material Type                       NaN
Procurement                         NaN
Input                               NaN
Output                              NaN
Name: 0, dtype: object

In [17]:
step_one.rename(columns={
    'Reference': 'Order Category',
    'Unnamed: 13': 'Order Data',
    'Unnamed: 14': 'Master Category',
    'Unnamed: 15': 'Master Data',
    'Unnamed: 16': 'Remark'
}, inplace=True)

In [18]:
step_one = step_one.iloc[1: , :]

In [19]:
step_one

,Unnamed: 0,Plant,Entry Date,Material,Location,Stock Type,Movement Type,Posting Date,Unit,Quantity,Material Document No,Provide,Order Category,Order Data,Master Category,Master Data,Remark,Previous Stock,Entry User,Material Type,Procurement,Input,Output
1,1.0,DYKV,07/01/2021 23:59:24,BN60-00162C,WPAS,Unrestricted,261 (GI for production order),07/01/2021,M,-39.0,1003609213-16,,Production Order,6500297936,,,,783.06,MES_USER,ROH,F,-39.0,-39.0
2,2.0,DYKV,07/01/2021 23:59:24,BN61-13557P,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-60.0,1003609213-15,,Production Order,6500297936,,,,78043.00,MES_USER,ROH,F,-60.0,-60.0
3,3.0,DYKV,07/01/2021 23:59:24,BN61-16157A,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-30.0,1003609213-14,,Production Order,6500297936,,,,595.00,MES_USER,ROH,F,-30.0,-30.0
4,4.0,DYKV,07/01/2021 23:59:24,BN96-46032A,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-60.0,1003609213-13,,Production Order,6500297936,,,,1068.00,MES_USER,ROH,F,-60.0,-60.0
5,5.0,DYKV,07/01/2021 23:59:24,BN63-17877A,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-30.0,1003609213-12,,Production Order,6500297936,,,,3057.00,MES_USER,ROH,F,-30.0,-30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21668,4051.0,DYKV,07/01/2021 00:20:59,PZ64-04335A,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,50.0,1003607406-1,,Production Order,6500297552,,,,119.00,MES_USER,HALB,E,50.0,50.0
21669,4052.0,DYKV,07/01/2021 00:20:27,IZ64-04516B,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,30.0,1003607405-1,,Production Order,6500297647,,,,488.00,MES_USER,HALB,E,30.0,30.0
21670,4053.0,DYKV,07/01/2021 00:13:27,IZ64-04514A,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,80.0,1003607404-1,,Production Order,6500297548,,,,7050.00,MES_USER,HALB,E,80.0,80.0
21671,4054.0,DYKV,07/01/2021 00:12:41,BN96-50471A,FPRE,Unrestricted,101 (GR for Order),07/01/2021,PC,20.0,1003607403-1,,Production Order,6500297708,,,,465.00,MES_USER,FERT,E,20.0,20.0


In [20]:
# step_one.loc[step_one['Input'] > 0]
def temp_filter(values):
    return values if values > 0 else 0

In [21]:
step_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21672 entries, 1 to 21672
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            21672 non-null  float64
 1   Plant                 21672 non-null  object 
 2   Entry Date            21672 non-null  object 
 3   Material              21672 non-null  object 
 4   Location              21672 non-null  object 
 5   Stock Type            21672 non-null  object 
 6   Movement Type         21672 non-null  object 
 7   Posting Date          21672 non-null  object 
 8   Unit                  21672 non-null  object 
 9   Quantity              21672 non-null  float64
 10  Material Document No  21672 non-null  object 
 11  Provide               21672 non-null  object 
 12  Order Category        21672 non-null  object 
 13  Order Data            21672 non-null  object 
 14  Master Category       21672 non-null  object 
 15  Master Data        

In [22]:
# def filter_inout(row):
#     return row['Quantity'] if row['Quantity'] > 0 else 0

In [23]:
# step_one['input'] = step_one.apply(filter_inout, axis=1)

In [24]:
pd.set_option('mode.chained_assignment', None)

In [25]:
step_one.loc[step_one['Quantity'] > 0, 'Input'] = step_one['Quantity']
step_one.loc[step_one['Quantity'] > 0, 'Output'] = 0
step_one.loc[step_one['Quantity'] <= 0, 'Input'] = 0
step_one.loc[step_one['Quantity'] <= 0, 'Output'] = -step_one['Quantity']

In [26]:
cols = step_one.columns.tolist()

In [27]:
cols

['Unnamed: 0',
 'Plant',
 'Entry Date',
 'Material',
 'Location',
 'Stock Type',
 'Movement Type',
 'Posting Date',
 'Unit',
 'Quantity',
 'Material Document No',
 'Provide',
 'Order Category',
 'Order Data',
 'Master Category',
 'Master Data',
 'Remark',
 'Previous Stock',
 'Entry User',
 'Material Type',
 'Procurement',
 'Input',
 'Output']

In [28]:
cols = cols[:10] + [cols[-2], cols[-1]] + cols[10:-2]

In [29]:
step_one = step_one[cols]

In [30]:
step_one['Account code'] = [int(x) for x in (step_one['Movement Type'].str[:3])]

In [31]:
step_one

,Unnamed: 0,Plant,Entry Date,Material,Location,Stock Type,Movement Type,Posting Date,Unit,Quantity,Input,Output,Material Document No,Provide,Order Category,Order Data,Master Category,Master Data,Remark,Previous Stock,Entry User,Material Type,Procurement,Account code
1,1.0,DYKV,07/01/2021 23:59:24,BN60-00162C,WPAS,Unrestricted,261 (GI for production order),07/01/2021,M,-39.0,0.0,39.0,1003609213-16,,Production Order,6500297936,,,,783.06,MES_USER,ROH,F,261
2,2.0,DYKV,07/01/2021 23:59:24,BN61-13557P,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-60.0,0.0,60.0,1003609213-15,,Production Order,6500297936,,,,78043.00,MES_USER,ROH,F,261
3,3.0,DYKV,07/01/2021 23:59:24,BN61-16157A,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-30.0,0.0,30.0,1003609213-14,,Production Order,6500297936,,,,595.00,MES_USER,ROH,F,261
4,4.0,DYKV,07/01/2021 23:59:24,BN96-46032A,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-60.0,0.0,60.0,1003609213-13,,Production Order,6500297936,,,,1068.00,MES_USER,ROH,F,261
5,5.0,DYKV,07/01/2021 23:59:24,BN63-17877A,WPAS,Unrestricted,261 (GI for production order),07/01/2021,PC,-30.0,0.0,30.0,1003609213-12,,Production Order,6500297936,,,,3057.00,MES_USER,ROH,F,261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21668,4051.0,DYKV,07/01/2021 00:20:59,PZ64-04335A,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,50.0,50.0,0.0,1003607406-1,,Production Order,6500297552,,,,119.00,MES_USER,HALB,E,101
21669,4052.0,DYKV,07/01/2021 00:20:27,IZ64-04516B,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,30.0,30.0,0.0,1003607405-1,,Production Order,6500297647,,,,488.00,MES_USER,HALB,E,101
21670,4053.0,DYKV,07/01/2021 00:13:27,IZ64-04514A,WPAS,Unrestricted,101 (GR for Order),07/01/2021,PC,80.0,80.0,0.0,1003607404-1,,Production Order,6500297548,,,,7050.00,MES_USER,HALB,E,101
21671,4054.0,DYKV,07/01/2021 00:12:41,BN96-50471A,FPRE,Unrestricted,101 (GR for Order),07/01/2021,PC,20.0,20.0,0.0,1003607403-1,,Production Order,6500297708,,,,465.00,MES_USER,FERT,E,101


In [32]:
cols = step_one.columns.tolist()

In [33]:
cols = cols[:7] + [cols[-1]] + cols[7: -1]

In [34]:
step_one = step_one[cols]

In [35]:
step_one

,Unnamed: 0,Plant,Entry Date,Material,Location,Stock Type,Movement Type,Account code,Posting Date,Unit,Quantity,Input,Output,Material Document No,Provide,Order Category,Order Data,Master Category,Master Data,Remark,Previous Stock,Entry User,Material Type,Procurement
1,1.0,DYKV,07/01/2021 23:59:24,BN60-00162C,WPAS,Unrestricted,261 (GI for production order),261,07/01/2021,M,-39.0,0.0,39.0,1003609213-16,,Production Order,6500297936,,,,783.06,MES_USER,ROH,F
2,2.0,DYKV,07/01/2021 23:59:24,BN61-13557P,WPAS,Unrestricted,261 (GI for production order),261,07/01/2021,PC,-60.0,0.0,60.0,1003609213-15,,Production Order,6500297936,,,,78043.00,MES_USER,ROH,F
3,3.0,DYKV,07/01/2021 23:59:24,BN61-16157A,WPAS,Unrestricted,261 (GI for production order),261,07/01/2021,PC,-30.0,0.0,30.0,1003609213-14,,Production Order,6500297936,,,,595.00,MES_USER,ROH,F
4,4.0,DYKV,07/01/2021 23:59:24,BN96-46032A,WPAS,Unrestricted,261 (GI for production order),261,07/01/2021,PC,-60.0,0.0,60.0,1003609213-13,,Production Order,6500297936,,,,1068.00,MES_USER,ROH,F
5,5.0,DYKV,07/01/2021 23:59:24,BN63-17877A,WPAS,Unrestricted,261 (GI for production order),261,07/01/2021,PC,-30.0,0.0,30.0,1003609213-12,,Production Order,6500297936,,,,3057.00,MES_USER,ROH,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21668,4051.0,DYKV,07/01/2021 00:20:59,PZ64-04335A,WPAS,Unrestricted,101 (GR for Order),101,07/01/2021,PC,50.0,50.0,0.0,1003607406-1,,Production Order,6500297552,,,,119.00,MES_USER,HALB,E
21669,4052.0,DYKV,07/01/2021 00:20:27,IZ64-04516B,WPAS,Unrestricted,101 (GR for Order),101,07/01/2021,PC,30.0,30.0,0.0,1003607405-1,,Production Order,6500297647,,,,488.00,MES_USER,HALB,E
21670,4053.0,DYKV,07/01/2021 00:13:27,IZ64-04514A,WPAS,Unrestricted,101 (GR for Order),101,07/01/2021,PC,80.0,80.0,0.0,1003607404-1,,Production Order,6500297548,,,,7050.00,MES_USER,HALB,E
21671,4054.0,DYKV,07/01/2021 00:12:41,BN96-50471A,FPRE,Unrestricted,101 (GR for Order),101,07/01/2021,PC,20.0,20.0,0.0,1003607403-1,,Production Order,6500297708,,,,465.00,MES_USER,FERT,E
